# Generators
***
A generator in python is a function that returns a set of items one at a time. In other words, you can iterate over the functions and retrieve each object.

Generators look exactly like functions but instead of returning elements, you `yield` elements.

In [37]:
def range_gen(n=3):
    i = 0
    while i < n:
        yield i
        i += 1

In [44]:
range_5 = range_gen(5)
for i in range_5:
    print(i, end=' ')
    
print()
# or just
for i in range_gen(5):
    print(i, end=' ')

0 1 2 3 4 
0 1 2 3 4 

Above we defined our first generator, `range_gen`. All it does, is feed us the numbers 0 to n, one at a time. 

Every time we call range_5 in the for loop, the generator **yields** the next value. This tells us that it maintains some sort of state remembering where it is. But why do we care? Why not just return the entire list of numbers from 0 to n and iterate over that in the for loop?



In [47]:
range_5
print(next(range_5))
print(next(range_5))
print(next(range_5))
print(next(range_5))

StopIteration: 